In [ ]:
import requests
from bs4 import BeautifulSoup
import pickle

base = 'https://courses.illinois.edu/cisapp/explorer/catalog'
year, term, subject, courseNumber = ('',)*4

def get_Text_from_Tag(tag):
  url = base
  if year != '':
    url += '/' + year
  if term != '':
    url += '/' + term
  if subject != '':
    url += '/' + subject
  if courseNumber != '':
    url += '/' + courseNumber
  url += '.xml'
  print(url)
  page = requests.get(url).text
  soup = BeautifulSoup(page, "xml")
  return soup.findAll(tag)

yearArray = get_Text_from_Tag('calendarYear')
for currentYear in yearArray:
  year = currentYear.string
  if year == '2024' or year == '2023':
    continue
  print(year)
  termArray = get_Text_from_Tag('term')
  for currentTerm in termArray:
    term = currentTerm.string.split()[0].lower()
    print(term)
    subjectArray = get_Text_from_Tag('subject')
    for currentSubject in subjectArray:
      subject = currentSubject.get('id')
      print(subject)
      courseNumberArray = get_Text_from_Tag('course')
      for currentCourseNumber in courseNumberArray:
        courseNumber = currentCourseNumber.get('id')
        #finally can access all info for a specific class
        print(get_Text_from_Tag('label')[0].string)
        courseNumber = ''
      subject = ''
    term = ''


https://courses.illinois.edu/cisapp/explorer/catalog.xml
2022
https://courses.illinois.edu/cisapp/explorer/catalog/2022.xml
winter
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter.xml
ACE
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE.xml
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/100.xml
Introduction to Applied Microeconomics
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/161.xml
Microcomputer Applications
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/199.xml
Undergraduate Open Seminar
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/203.xml
Introduction to Public Policy and Law
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/210.xml
Environmental Economics
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/222.xml
Agricultural Marketing
https://courses.illinois.edu/cisapp/explorer/catalog/2022/winter/ACE/231.xml
Food a

KeyboardInterrupt: ignored